In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/fundos-imobiliarios/busca-avancada')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/funds.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [10]:
fundinvest = pd.read_csv('funds.csv', sep=';',  decimal=',', thousands='.', encoding='ISO-8859-1' )
#fiis = pd.DataFrame(funds)


fundinvest = fundinvest.loc[:, ['TICKER', 'PRECO', 'GESTAO' , 'DY', 'P/VP', 'PERCENTUAL EM CAIXA', 
                        'N COTISTAS', 'CAGR DIVIDENDOS 3 ANOS', ' CAGR VALOR CORA 3 ANOS', 
                        'LIQUIDEZ MEDIA DIARIA', 'PATRIMONIO', 'VALOR PATRIMONIAL COTA', 
                        ' N COTAS', 'ULTIMO DIVIDENDO']]

fundinvest.fillna(value=0, inplace=True)

fundinvest.rename(columns={'PERCENTUAL EM CAIXA': "PERC_CX" }, inplace=True)
fundinvest.rename(columns={'CAGR DIVIDENDOS 3 ANOS': "DY_CAGR_3ANOS" }, inplace=True)
fundinvest.rename(columns={' CAGR VALOR CORA 3 ANOS': "VLR_CAGR_3ANOS" }, inplace=True)
fundinvest.rename(columns={'LIQUIDEZ MEDIA DIARIA': "LIQ_DIARIA" }, inplace=True)
fundinvest.rename(columns={'VALOR PATRIMONIAL COTA': "VLR_PATR_COTA" }, inplace=True)
fundinvest.rename(columns={'ULTIMO DIVIDENDO': "ULT_DIV" }, inplace=True)

fundinvest.rename(columns={'TICKER': "TICKERS" }, inplace=True)

fundinvest

,TICKERS,PRECO,GESTAO,DY,P/VP,PERC_CX,N COTISTAS,DY_CAGR_3ANOS,VLR_CAGR_3ANOS,LIQ_DIARIA,PATRIMONIO,VLR_PATR_COTA,N COTAS,ULT_DIV
0,ABCP11,67.89,Passiva,8.35,0.74,1.53,17906.0,13.18,-4.52,69929.48,4.325528e+08,91.86,4709082.0,0.500000
1,AFHI11,97.80,Ativa,12.90,1.02,2.10,22313.0,0.00,0.00,1187180.65,2.897112e+08,95.95,3019453.0,1.050000
2,AIEC11,68.79,Ativa,13.20,0.73,2.54,15564.0,9.48,0.00,733822.06,4.573551e+08,94.79,4824987.0,0.760000
3,ALMI11,736.00,Passiva,0.00,0.36,2.24,2367.0,0.00,-14.66,50133.59,2.257938e+08,2030.94,111177.0,0.371133
4,ALZC11,0.00,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.000000e+00,0.00,1000000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,XPPR11,22.34,Ativa,9.85,0.32,3.67,61991.0,-30.77,-35.97,336843.77,5.156974e+08,70.49,7316171.0,0.100000
467,XPSF11,8.57,Ativa,10.69,0.97,1.24,46486.0,0.78,-3.54,582819.26,3.816852e+08,8.81,43302140.0,0.075000
468,YUFI11,88.90,Ativa,3.70,0.87,8.94,56.0,0.00,0.00,149.00,5.008454e+07,101.66,492650.0,0.383187
469,ZAVI11,129.35,Ativa,9.53,0.96,6.33,3311.0,0.00,0.00,377343.13,1.586591e+08,135.22,1173322.0,1.160000


In [6]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = fundinvest['TICKERS']
list_funds = ticker.to_list()

indicators_list_funds = []
for funds in list_funds:

    #Criando url
    url = f"https://statusinvest.com.br/fundos-imobiliarios/{funds}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        NAME = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/h1/small').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[3]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[5]/div/div[1]/strong').text
        TIPO = navegador.find_element(By.XPATH, '//*[@id="fund-section"]/div/div/div[2]/div/div[6]/div/div/strong').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="fund-section"]/div/div/div[2]/div/div[6]/div/div/strong').text
        
        

        #Criando dict
        dicionario = {  "funds": funds,
                        "NAME": NAME,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "TIPO": TIPO,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        indicators_list_funds.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()

In [7]:
funds = pd.DataFrame.from_dict(indicators_list_funds)

funds['MIN_MES'] = (funds['MIN_MES']).str.lstrip('$')
funds['MAX_MES'] = (funds['MAX_MES']).str.lstrip('$')
funds['VAL_12M'] = (funds['VAL_12M']).str.rstrip('%')
funds['VAL_MES'] = (funds['VAL_MES']).str.rstrip('%')

funds.to_csv('list_funds.csv')

list_funds = pd.read_csv("list_funds.csv", sep=',', decimal=',', thousands='.', encoding='ISO-8859-1' )

list_funds.fillna(value=0, inplace=True)
list_funds = list_funds.drop(['Unnamed: 0'],axis=1)
list_funds.rename(columns={'funds': "TICKERS" }, inplace=True)

list_funds

,TICKERS,NAME,MIN_MES,MAX_MES,MIN_52S,MAX_52S,VAL_12M,VAL_MES,TIPO,SEGMENTO
0,ABCP11,GRAND PLAZA SHOPPING,"R$ 67,10","R$ 70,00","63,99","76,10","-10,54","-10,54",Shoppings,Shoppings
1,AFHI11,AF INVEST CRI,"R$ 97,00","R$ 97,80","92,17","100,31","-2,50","-2,50",TÃ­tulos e Valores MobiliÃ¡rios,TÃ­tulos e Valores MobiliÃ¡rios
2,AIEC11,AUTONOMY EDIFÃCIOS CORPORATIVOS,"R$ 65,83","R$ 68,98","59,48","81,00","-12,09","-12,09",Lajes Corporativas,Lajes Corporativas
3,ALMI11,TORRE ALMIRANTE,"R$ 714,42","R$ 773,95","714,42","1.000,00","-18,03","-18,03",Lajes Corporativas,Lajes Corporativas
4,ALZC11,ALIANZA CRÃDITO IMOBILIÃRIO FII FUNDO DE INV...,R$ -,R$ -,-,-,-,-,0,0
...,...,...,...,...,...,...,...,...,...,...
466,XPPR11,XP PROPERTIES FDO. INV. IMOB.,"R$ 22,32","R$ 22,79","22,03","46,55","-52,01","-52,01",Outros,Outros
467,XPSF11,XP SELECTION FDO DE FUNDOS INV IMOB - FII,"R$ 8,42","R$ 8,68","6,67","8,75","12,76","12,76",TÃ­tulos e Valores MobiliÃ¡rios,TÃ­tulos e Valores MobiliÃ¡rios
468,YUFI11,YUCA FDO INV. IMOB.,"R$ 88,90","R$ 89,99","63,01","109,96","-19,15","-19,15",Residencial,Residencial
469,ZAVI11,ZAVIT REAL ESTATE FUND,"R$ 123,97","R$ 129,95","116,00","136,00","-0,50","-0,50",HÃ­brido,HÃ­brido


In [14]:
invest_funds = pd.merge(fundinvest, list_funds, on='TICKERS')

invest_funds = invest_funds[['TICKERS', 'NAME', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M',
                             'GESTAO' , 'DY', 'P/VP', 'PERC_CX', 'N COTISTAS', 'DY_CAGR_3ANOS', 
                             'VLR_CAGR_3ANOS', 'LIQ_DIARIA', 'PATRIMONIO', 'VLR_PATR_COTA', 
                             ' N COTAS', 'ULT_DIV', 'TIPO', 'SEGMENTO']]

invest_funds

,TICKERS,NAME,PRECO,MIN_MES,MAX_MES,VAL_MES,MIN_52S,MAX_52S,VAL_12M,GESTAO,...,N COTISTAS,DY_CAGR_3ANOS,VLR_CAGR_3ANOS,LIQ_DIARIA,PATRIMONIO,VLR_PATR_COTA,N COTAS,ULT_DIV,TIPO,SEGMENTO
0,ABCP11,GRAND PLAZA SHOPPING,67.89,"R$ 67,10","R$ 70,00","-10,54","63,99","76,10","-10,54",Passiva,...,17906.0,13.18,-4.52,69929.48,4.325528e+08,91.86,4709082.0,0.500000,Shoppings,Shoppings
1,AFHI11,AF INVEST CRI,97.80,"R$ 97,00","R$ 97,80","-2,50","92,17","100,31","-2,50",Ativa,...,22313.0,0.00,0.00,1187180.65,2.897112e+08,95.95,3019453.0,1.050000,TÃ­tulos e Valores MobiliÃ¡rios,TÃ­tulos e Valores MobiliÃ¡rios
2,AIEC11,AUTONOMY EDIFÃCIOS CORPORATIVOS,68.79,"R$ 65,83","R$ 68,98","-12,09","59,48","81,00","-12,09",Ativa,...,15564.0,9.48,0.00,733822.06,4.573551e+08,94.79,4824987.0,0.760000,Lajes Corporativas,Lajes Corporativas
3,ALMI11,TORRE ALMIRANTE,736.00,"R$ 714,42","R$ 773,95","-18,03","714,42","1.000,00","-18,03",Passiva,...,2367.0,0.00,-14.66,50133.59,2.257938e+08,2030.94,111177.0,0.371133,Lajes Corporativas,Lajes Corporativas
4,ALZC11,ALIANZA CRÃDITO IMOBILIÃRIO FII FUNDO DE INV...,0.00,R$ -,R$ -,-,-,-,-,0,...,0.0,0.00,0.00,0.00,0.000000e+00,0.00,1000000.0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,XPPR11,XP PROPERTIES FDO. INV. IMOB.,22.34,"R$ 22,32","R$ 22,79","-52,01","22,03","46,55","-52,01",Ativa,...,61991.0,-30.77,-35.97,336843.77,5.156974e+08,70.49,7316171.0,0.100000,Outros,Outros
467,XPSF11,XP SELECTION FDO DE FUNDOS INV IMOB - FII,8.57,"R$ 8,42","R$ 8,68","12,76","6,67","8,75","12,76",Ativa,...,46486.0,0.78,-3.54,582819.26,3.816852e+08,8.81,43302140.0,0.075000,TÃ­tulos e Valores MobiliÃ¡rios,TÃ­tulos e Valores MobiliÃ¡rios
468,YUFI11,YUCA FDO INV. IMOB.,88.90,"R$ 88,90","R$ 89,99","-19,15","63,01","109,96","-19,15",Ativa,...,56.0,0.00,0.00,149.00,5.008454e+07,101.66,492650.0,0.383187,Residencial,Residencial
469,ZAVI11,ZAVIT REAL ESTATE FUND,129.35,"R$ 123,97","R$ 129,95","-0,50","116,00","136,00","-0,50",Ativa,...,3311.0,0.00,0.00,377343.13,1.586591e+08,135.22,1173322.0,1.160000,HÃ­brido,HÃ­brido
